# 作成した教材をバックアップします
<HR>

ホームディレクトリで作成した教材をバックアップします。

# バックアップの準備

このNotebookを実行すると、ホームディレクトリ直下の下記のフォルダをバックアップファイルに保存します。

**バックアップしたいファイル/フォルダは、下記のフォルダに移動してください。**

* `textbook`
  + 教材Notebookおよび教材Notebookから呼び出すスクリプトなど
* `info`
  + 受講者が参照する参考資料など
* `draft`
  + 教材以外で保存して置きたいNotebook、その他ファイル

注） バックアップファイルには `test` フォルダも保存されます。
* `test` フォルダは、この教材の自動テストに必要なファイルを保存するフォルダです。<br>（初期状態で、テスト手順Notebookのテンプレートが配置されています）
* **`test` フォルダにはその他のファイルを格納しないで下さい。**

# その他にバックアップしたいファイル/フォルダがある場合

もし、`textbook`、`info`、`draft` 以外に保存したいファイルまたはフォルダがある場合は、`INCLUDES` にスペース区切りで列挙して下さい。
* `textbook`、`info`、`draft`、`test` フォルダは必ずバックアップに保存されます。
* 管理用Notebookフォルダ `tools` はバックアップに保存されません。
* ホームディレクトリからの相対パスで指定します。
```
%env INCLUDES=ext-textbook  ext-data  Sample.ipynb
```

**次のセルでバックアップに含めたいファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
%env INCLUDES=

# バックアップに含めないファイル/フォルダを指定します

バックアップファイルに保存しないファイルまたはフォルダがある場合は、EXCLUDES にスペース区切りで列挙して下さい。
* 管理用Notebookフォルダ「admin_tools」はバックアップファイルに保存されません
```
EXCLUDES=".ipynb_checkpoints"
```

**次のセルで除外するファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
%env EXCLUDES=.ipynb_checkpoints .log

# コースの指定

In [ ]:
import os

# コース名の指定
course_name = os.environ['CWH_COURSE_NAME']
course_name

# バックアップの作業用フォルダを作成します

* バックアップファイルを格納するフォルダを作成します。
* フォルダを変更する場合は、BACKUP_WORK_DIR を変更して下さい。
* フォルダが既存の場合、フォルダ配下のファイルは削除されます。必要があれば事前に退避して下さい。

**次のセルを実行して、作業用フォルダを作成して下さい。**

In [ ]:
%env BACKUP_WORK_DIR=~/{course_name}/backup

import os.path

backupdir = os.path.expanduser(os.environ['BACKUP_WORK_DIR'])
print(backupdir)
%env BACKUP_WORK_DIR={backupdir}

# create backup work directory
if not os.path.exists(backupdir):
    os.makedirs(backupdir)

# cleanup backup work directory
!rm -rf {backupdir}/*

# バックアップ名を指定します

* バックパップの名前を BACKUP_NAME に設定して下さい。
```
BACKUP_NAME="course"
```
* バックアップファイルは、「バックアップ名_backup.tar.gz」として保存します。

**次のセルでバックアップ名を指定して、セルを実行して下さい。**

In [ ]:
%env BACKUP_NAME=course-01

# 次のセルを実行して、バックアップファイルを作成して下さい

* バックアップファイルを生成します。
* バックアップファイルは、上記で作成したバックアップフォルダに格納されます。

In [ ]:
%%bash
set -e

TARGET_FILES=('textbook' 'info' 'draft' 'test' ${INCLUDES})
EXCLUDE_PATTERNS=('backup-*.tgz' '.ipynb_checkpoints' ${EXCLUDES})
BACKUP_NAME="${BACKUP_NAME,,}"
[ -z "${BACKUP_NAME}" ] && BACKUP_NAME="$(< /etc/courseware.configmap/fqdn)"
[ -z "${BACKUP_NAME}" ] && BACKUP_NAME="untitled"
TMP_WORK_DIR="$(mktemp -d --tmpdir courseware-bk.tmp.XXXXXXXX)"
TGZ_FILE="${TMP_WORK_DIR}/backup-${BACKUP_NAME}.tgz"
TARGET_LIST="${TMP_WORK_DIR}/target.list"
EXCLUDE_LIST="${TMP_WORK_DIR}/exclude.list"

cd ~/${CWH_COURSE_NAME}
for entry in "${TARGET_FILES[@]}"; do
  if [ -e "${entry}" ]; then
    echo "$(realpath --relative-base=. "${entry}")" >>"${TARGET_LIST}"
  fi
done
for entry in "${EXCLUDE_PATTERNS[@]}"; do
  echo ${entry} >>"${EXCLUDE_LIST}"
done

tar -cz -X <(sort -u "${EXCLUDE_LIST}") -T <(sort -u "${TARGET_LIST}") -f "${TGZ_FILE}"
mv "${TGZ_FILE}" ~/${CWH_COURSE_NAME}/
rm -rf "${TMP_WORK_DIR}"

TGZ_NAME=$(basename "${TGZ_FILE}")
TGZ_DST_PATH=$(realpath ~/"${CWH_COURSE_NAME}"/"${TGZ_NAME}")

echo 'Backup completed successfully:' "${TGZ_DST_PATH}"

# バックアップファイルを保存します

* バックアップファイルを適切な保管場所へ転送して保存して下さい。
* ローカルマシンに保存する場合は、下記の手順でダウンロードできます。
  1. ブラウザのHome画面で、バックアップファイルを格納したフォルダを開きます。バックアップファイルが表示されます。
  2. バックアップファイル名先頭のチェックボックスをクリックします。画面左上に操作ボタンが表示されます。
  3. [Download]ボタンをクリックします。